In [34]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()

True

In [35]:
stones = pd.read_csv("../data/albumlist.csv",  index_col=0, encoding='latin-1')

In [36]:
stones
stones.info()
stones.isna().sum() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1 to 500
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Year      500 non-null    int64 
 1   Album     500 non-null    object
 2   Artist    500 non-null    object
 3   Genre     500 non-null    object
 4   Subgenre  500 non-null    object
dtypes: int64(1), object(4)
memory usage: 23.4+ KB


Year        0
Album       0
Artist      0
Genre       0
Subgenre    0
dtype: int64

In [37]:
stones.Year.unique()

array([1967, 1966, 1965, 1971, 1972, 1979, 1968, 1976, 1959, 1969, 1975,
       1991, 1982, 1990, 1970, 1973, 1963, 1977, 1987, 2001, 1984, 1988,
       1957, 1964, 1956, 2000, 1986, 1980, 1978, 1955, 1962, 2003, 1995,
       1974, 2005, 1960, 1994, 1992, 1985, 2004, 1958, 1989, 1997, 2002,
       1998, 1983, 2006, 1981, 1961, 1999, 1993, 1996, 2007, 2010, 2011,
       2008])

In [38]:
stones.Album.unique()

array(["Sgt. Pepper's Lonely Hearts Club Band", 'Pet Sounds', 'Revolver',
       'Highway 61 Revisited', 'Rubber Soul', "What's Going On",
       'Exile on Main St.', 'London Calling', 'Blonde on Blonde',
       'The Beatles ("The White Album")', 'The Sun Sessions',
       'Kind of Blue', 'The Velvet Underground & Nico', 'Abbey Road',
       'Are You Experienced', 'Blood on the Tracks', 'Nevermind',
       'Born to Run', 'Astral Weeks', 'Thriller',
       'The Great Twenty_Eight', 'The Complete Recordings',
       'John Lennon/Plastic Ono Band', 'Innervisions',
       'Live at the Apollo, 1962', 'Rumours', 'The Joshua Tree',
       "Who's Next", 'Led Zeppelin', 'Blue', 'Bringing It All Back Home',
       'Let It Bleed', 'Ramones', 'Music From Big Pink',
       'The Rise and Fall of Ziggy Stardust and the Spiders From Mars',
       'Tapestry', 'Hotel California', 'The Anthology',
       'Please Please Me', 'Forever Changes',
       "Never Mind the Bollocks Here's the Sex Pistols", 'The 

In [39]:
stones.Genre.value_counts()

Rock                                         249
Funk / Soul                                   38
Hip Hop                                       29
Electronic, Rock                              19
Rock, Pop                                     18
                                            ... 
Hip Hop, Rock, Funk / Soul                     1
Electronic, Rock, Funk / Soul, Pop             1
Rock, Funk / Soul, Folk, World, & Country      1
Rock,ÊBlues                                    1
Rock, Reggae, Latin                            1
Name: Genre, Length: 63, dtype: int64

In [40]:
stones["Gen"] = stones["Genre"].str.extract(r"(^([^,])+)")[0]

In [41]:
stones.Gen.value_counts()

Rock           318
Funk / Soul     51
Electronic      45
Hip Hop         34
Jazz            19
Folk            13
Blues            9
Reggae           7
Pop              2
Classical        1
Latin            1
Name: Gen, dtype: int64

In [42]:
stones["Subgen"] = stones["Subgenre"].str.extract(r"(^([^,])+)")[0]

In [43]:
stones.Subgen.unique()

array(['Rock & Roll', 'Pop Rock', 'Psychedelic Rock', 'Folk Rock', 'Soul',
       'Blues Rock', 'Punk', 'Modal', 'Garage Rock', 'Alternative Rock',
       'Acoustic', 'Disco', 'Delta Blues', 'Rhythm & Blues', 'Hard Rock',
       'Classic Rock', 'Folk', 'Beat', 'Prog Rock', 'Art Rock',
       'Soundtrack', 'Reggae', 'Free Jazz', 'Conscious', 'Electric Blues',
       'None', 'Dialogue', 'Doo Wop', 'Country', 'Ska', 'Fusion',
       'New Wave', 'Big Band', 'Hard Bop', 'Soft Rock', 'Funk',
       'Roots Reggae', 'Thug Rap', 'Indie Rock', 'Gangsta', 'Bayou Funk',
       'Chicago Blues', 'Soul-Jazz', 'Jazz-Rock', 'Afro-Cuban',
       'Post-Punk', 'Glam', 'Thrash', 'Power Pop', 'P.Funk', 'Europop',
       'Synth-pop', 'Country Rock', 'Acid Rock', 'Industrial', 'Grunge',
       'Piano Blues', 'Mod', 'Pop Rap', 'Country Blues',
       'Musique Concr?te', 'Heavy Metal', 'Electro', 'RnB/Swing', 'Vocal',
       'Noise', 'Cajun', 'Avantgarde', 'Modern Classical', 'Lo-Fi',
       'Contemporary Jazz'

In [44]:
stones["Subgen"] = stones["Subgen"].str.replace("Musique concrète ?te","Musique concrète", regex = True)

In [45]:
stones.Subgen.unique()

array(['Rock & Roll', 'Pop Rock', 'Psychedelic Rock', 'Folk Rock', 'Soul',
       'Blues Rock', 'Punk', 'Modal', 'Garage Rock', 'Alternative Rock',
       'Acoustic', 'Disco', 'Delta Blues', 'Rhythm & Blues', 'Hard Rock',
       'Classic Rock', 'Folk', 'Beat', 'Prog Rock', 'Art Rock',
       'Soundtrack', 'Reggae', 'Free Jazz', 'Conscious', 'Electric Blues',
       'None', 'Dialogue', 'Doo Wop', 'Country', 'Ska', 'Fusion',
       'New Wave', 'Big Band', 'Hard Bop', 'Soft Rock', 'Funk',
       'Roots Reggae', 'Thug Rap', 'Indie Rock', 'Gangsta', 'Bayou Funk',
       'Chicago Blues', 'Soul-Jazz', 'Jazz-Rock', 'Afro-Cuban',
       'Post-Punk', 'Glam', 'Thrash', 'Power Pop', 'P.Funk', 'Europop',
       'Synth-pop', 'Country Rock', 'Acid Rock', 'Industrial', 'Grunge',
       'Piano Blues', 'Mod', 'Pop Rap', 'Country Blues',
       'Musique Concr?te', 'Heavy Metal', 'Electro', 'RnB/Swing', 'Vocal',
       'Noise', 'Cajun', 'Avantgarde', 'Modern Classical', 'Lo-Fi',
       'Contemporary Jazz'

In [46]:
stones.Subgen.value_counts().head(5)

Pop Rock            43
Folk Rock           40
Alternative Rock    38
Blues Rock          37
None                29
Name: Subgen, dtype: int64

In [47]:
stones.sample(5)

,Year,Album,Artist,Genre,Subgenre,Gen,Subgen
Number,,,,,,,
2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock",Rock,Pop Rock
23,1970,John Lennon/Plastic Ono Band,John Lennon / Plastic Ono Band,Rock,Pop Rock,Rock,Pop Rock
217,1961,Two Steps From the Blues,"Bobby ""Blue"" Bland","Funk / Soul, Blues","Rhythm & Blues, Soul",Funk / Soul,Rhythm & Blues
313,1994,MTV Unplugged in New York,Nirvana,Rock,"Folk Rock, Acoustic, Grunge",Rock,Folk Rock
490,1973,Tres hombres,ZZ Top,Rock,"Blues Rock, Classic Rock",Rock,Blues Rock


In [49]:
stones.to_csv("../data/stones.csv")